In [1]:
import pandas as pd


In [2]:
# Load the datasets
train_df = pd.read_csv("/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Assignment/BioXAi/train.csv")
val_df = pd.read_csv("/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Assignment/BioXAi/val.csv")
test_df = pd.read_csv("/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Assignment/BioXAi/test.csv")

# Add a column to keep track of dataset source (optional but handy)
train_df["split"] = "train"
val_df["split"] = "val"
test_df["split"] = "test"

# Merge all into a single DataFrame
full_df = pd.concat([train_df, val_df, test_df], ignore_index=True)


In [3]:
# Perform one-hot encoding
one_hot_df = pd.get_dummies(full_df['SAMPLE_DATA_TYPE'], prefix='DATA_TYPE').astype(int)

In [4]:
full_df = pd.concat([full_df, one_hot_df], axis=1)

In [5]:
# Select concentration and inhibition columns
conc_cols = [f'CONC{i}' for i in range(15)]
inh_cols = [f'DATA{i}' for i in range(15)]

In [6]:
import numpy as np

In [7]:
# Create concentration and inhibition column names
conc_cols = [f'CONC{i}' for i in range(15)]
inh_cols = [f'DATA{i}' for i in range(15)]

# Melt inhibition and concentration together
long_df = full_df.melt(
    id_vars=['canonical_smiles'] + list(one_hot_df.columns), 
    value_vars=inh_cols + conc_cols,
    var_name='col_type', 
    value_name='value'
)

# Extract the type (CONC/DATA) and index (0–14)
long_df['type'] = long_df['col_type'].str.extract(r'([A-Z]+)')
long_df['index'] = long_df['col_type'].str.extract(r'(\d+)').astype(int)

# Pivot to long format
final_df = long_df.pivot_table(
    index=['canonical_smiles', 'index'] + list(one_hot_df.columns), 
    columns='type', values='value'
).reset_index()

# Rename columns
final_df.rename(columns={'CONC': 'concentration', 'DATA': 'inhibition'}, inplace=True)

# Apply log10 to concentration
final_df['log_conc'] = np.log10(final_df['concentration'].astype(float))

In [8]:
import torch 
graph_dataset = torch.load('/home/nilabjab/cancer_dependency_project_nilabja/cancer_dependency_project/Assignment/BioXAi/graph_dataset.pt')

/tmp/ipykernel_3692708/1060271207.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  graph_dataset = torch.load('/home/nilabjab/cancer_dependency_project_nilabja/cancer_dep

In [9]:
len(graph_dataset)

103518

In [10]:
# Assuming you have a list of SMILES that match the graph order
smiles_list = full_df['canonical_smiles'].tolist()

# Map SMILES to graph by position
smiles_to_graph = dict(zip(smiles_list, graph_dataset))

In [11]:
# Assign to final_df
final_df['graph'] = final_df['canonical_smiles'].map(smiles_to_graph)

In [12]:
final_df = final_df.drop(columns='index')


In [13]:
final_df['graph']

0         [(x, [tensor([5]), tensor([6]), tensor([6]), t...
1         [(x, [tensor([5]), tensor([6]), tensor([6]), t...
2         [(x, [tensor([5]), tensor([6]), tensor([6]), t...
3         [(x, [tensor([5]), tensor([6]), tensor([6]), t...
4         [(x, [tensor([5]), tensor([6]), tensor([6]), t...
                                ...                        
974535    [(x, [tensor([8]), tensor([34]), tensor([8]), ...
974536    [(x, [tensor([8]), tensor([34]), tensor([8]), ...
974537    [(x, [tensor([8]), tensor([34]), tensor([8]), ...
974538    [(x, [tensor([8]), tensor([34]), tensor([8]), ...
974539    [(x, [tensor([8]), tensor([34]), tensor([8]), ...
Name: graph, Length: 974540, dtype: object

In [14]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import torch
from torch_geometric.data import Data, Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

In [15]:
# --- 0. Helper function to parse your graph string ---
# This is a placeholder. You'll need to adapt this based on the actual
# structure of your 'graph0', 'graph1', etc. columns.
# For this example, I'll assume it means you have node features (x)
# and edge_index.
def parse_graph_string(graph_str):
    # Example: "[(x, [tensor([5]), tensor([6]), ...]), (edge_index, ...)]"
    # This is highly dependent on your actual string format.
    # You'll need to implement robust parsing here.
    # For now, let's assume a simplified scenario where you can extract
    # node features and edge connectivity directly or from SMILES.
    pass

In [22]:
# --- 1. Load Data ---

# Let's assume 'inhibition' is your target variable (response)
TARGET_COLUMN = 'inhibition'

# --- 2. SMILES to Graph Conversion (using RDKit) ---
# This function converts a SMILES string to a PyG Data object.
# Node features can be atomic properties. Edge features can represent bond types.

def smiles_to_pyg_graph(smiles, other_features, y):
    """
    Converts a SMILES string and other features to a PyG Data object.

    Args:
        smiles (str): The SMILES string of the molecule.
        other_features (torch.Tensor): Tensor of other tabular features.
        y (float): The target variable.

    Returns:
        torch_geometric.data.Data: A PyG Data object, or None if conversion fails.
    """
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    # --- Node Features (Atom Features) ---
    # Example: Atomic number, formal charge, hybridization, aromaticity, etc.
    atom_features = []
    for atom in mol.GetAtoms():
        atom_features.append([
            atom.GetAtomicNum(),
            atom.GetFormalCharge(),
            float(atom.GetHybridization()), # Convert HybridizationType to float
            int(atom.GetIsAromatic()),
            atom.GetDegree(), # Number of explicit and implicit Hs
            atom.GetTotalNumHs(),
            # Add more features as needed
        ])
    x = torch.tensor(atom_features, dtype=torch.float)

    if x.shape[0] == 0: # No atoms in molecule (e.g. empty smiles)
        return None

    # --- Edge Index (Adjacency List) and Edge Features (Bond Features) ---
    edge_indices = []
    edge_attrs = []
    for bond in mol.GetBonds():
        start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        edge_indices.append((start, end))
        edge_indices.append((end, start)) # Add edges in both directions for undirected graphs

        # Example: Bond type
        bond_type = bond.GetBondTypeAsDouble() # GetBondType() returns BondType, convert to numeric
        edge_attrs.append([bond_type])
        edge_attrs.append([bond_type])

    if not edge_indices: # If no bonds, create a self-loop for each node or handle as needed
        if x.shape[0] > 0:
            edge_index = torch.empty((2,0), dtype=torch.long) # No edges
            edge_attr = torch.empty((0,1), dtype=torch.float) # No edge features
        else: # no nodes, no edges
             return None # Or handle this case appropriately
    else:
        edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_attrs, dtype=torch.float)


    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr,
                other_features=other_features.unsqueeze(0), # Add batch dimension for concatenation later
                y=torch.tensor([y], dtype=torch.float))

In [23]:
final_df

,canonical_smiles,DATA_TYPE_agonist1,DATA_TYPE_agonist2,DATA_TYPE_agonist3,DATA_TYPE_antagonist1,DATA_TYPE_antagonist2,DATA_TYPE_antagonist3,DATA_TYPE_cell_red,DATA_TYPE_viability1,DATA_TYPE_viability2,DATA_TYPE_viability3,concentration,inhibition,log_conc,graph
0,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,0,0,0,0,0,0,0,0,0,1,5.899000e-10,-2.509096,-9.229222,"Data(x=[28, 1], edge_index=[2, 58])"
1,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,0,0,0,0,0,0,0,0,1,0,5.899000e-10,-17.185048,-9.229222,"Data(x=[28, 1], edge_index=[2, 58])"
2,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,0,0,0,0,0,0,0,1,0,0,5.899000e-10,-3.018927,-9.229222,"Data(x=[28, 1], edge_index=[2, 58])"
3,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,0,0,0,0,0,0,1,0,0,0,7.373000e-08,0.356070,-7.132356,"Data(x=[28, 1], edge_index=[2, 58])"
4,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,0,0,0,0,0,1,0,0,0,0,5.899000e-10,-1.710692,-9.229222,"Data(x=[28, 1], edge_index=[2, 58])"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974535,[O-][Se](=O)[O-].[Na+].[Na+],0,0,0,0,1,0,0,0,0,0,2.326000e-05,0.382924,-4.633390,"Data(x=[6, 1], edge_index=[2, 6])"
974536,[O-][Se](=O)[O-].[Na+].[Na+],0,0,0,1,0,0,0,0,0,0,2.326000e-05,0.518981,-4.633390,"Data(x=[6, 1], edge_index=[2, 6])"
974537,[O-][Se](=O)[O-].[Na+].[Na+],0,0,1,0,0,0,0,0,0,0,2.326000e-05,-0.451559,-4.633390,"Data(x=[6, 1], edge_index=[2, 6])"
974538,[O-][Se](=O)[O-].[Na+].[Na+],0,1,0,0,0,0,0,0,0,0,2.326000e-05,0.590535,-4.633390,"Data(x=[6, 1], edge_index=[2, 6])"


In [24]:
# --- 3. Prepare Data for PyG ---
data_list = []
# Define which columns are 'other_features' (non-graph, non-target features)
# These are your one-hot encoded columns and log_conc
other_feature_cols = [
    'DATA_TYPE_agonist1', 'DATA_TYPE_agonist2', 'DATA_TYPE_agonist3',
    'DATA_TYPE_antagonist1', 'DATA_TYPE_antagonist2', 'DATA_TYPE_antagonist3',
    'DATA_TYPE_cell_red', 'DATA_TYPE_viability1', 'DATA_TYPE_viability2', 'DATA_TYPE_viability3',
    'log_conc' # 'concentration' could also be used, or log_conc if it's more stable
]

for index, row in final_df.iterrows():
    smiles = row['canonical_smiles']
    y = row[TARGET_COLUMN]
    # Extract other features and convert to a tensor
    other_feats_values = row[other_feature_cols].values.astype(np.float32)
    other_features_tensor = torch.tensor(other_feats_values, dtype=torch.float)

    # If you have pre-computed graph features in 'graph0', 'graph1', etc.
    # you would parse them here instead of using smiles_to_pyg_graph directly,
    # or use them to augment the graph features from RDKit.
    # For this example, we regenerate from SMILES.
    # graph_str = row['graph0'] # or graph1 etc. based on your logic
    # parsed_x, parsed_edge_index = parse_graph_string(graph_str) # Implement this!

    graph_data = smiles_to_pyg_graph(smiles, other_features_tensor, y)
    if graph_data:
        data_list.append(graph_data)

print(f"Successfully converted {len(data_list)} molecules to graph data.")
if not data_list:
    raise ValueError("No graph data could be created. Check SMILES strings and feature extraction.")

[21:11:35] WARNING: not removing hydrogen atom without neighbors
[21:11:35] WARNING: not removing hydrogen atom without neighbors
[21:11:35] WARNING: not removing hydrogen atom without neighbors
[21:11:35] WARNING: not removing hydrogen atom without neighbors
[21:11:35] WARNING: not removing hydrogen atom without neighbors
[21:11:35] WARNING: not removing hydrogen atom without neighbors
[21:11:35] WARNING: not removing hydrogen atom without neighbors
[21:11:35] WARNING: not removing hydrogen atom without neighbors
[21:11:35] WARNING: not removing hydrogen atom without neighbors
[21:11:35] WARNING: not removing hydrogen atom without neighbors
[21:11:35] WARNING: not removing hydrogen atom without neighbors
[21:11:35] WARNING: not removing hydrogen atom without neighbors
[21:11:35] WARNING: not removing hydrogen atom without neighbors
[21:11:35] WARNING: not removing hydrogen atom without neighbors
[21:11:35] WARNING: not removing hydrogen atom without neighbors
[21:11:35] WARNING: not r

Successfully converted 974540 molecules to graph data.


In [25]:
# --- 4. Split Data ---
train_data, temp_data = train_test_split(data_list, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print(f"Number of training samples: {len(train_data)}")
print(f"Number of validation samples: {len(val_data)}")
print(f"Number of test samples: {len(test_data)}")

# At this point, each element in train_data, val_data, test_data is a PyG 'Data' object.
# Each 'Data' object contains:
# - data.x: Node features (from atoms)
# - data.edge_index: Graph connectivity
# - data.edge_attr: Edge features (from bonds)
# - data.other_features: Your one-hot encoded data and log_conc
# - data.y: The target variable ('inhibition')

Number of training samples: 779632
Number of validation samples: 97454
Number of test samples: 97454


In [27]:
from torch_geometric.loader import DataLoader # Corrected import

# Batch size
batch_size = 64 # Keep it small for this dummy data example

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Example of a batch:
# for batch in train_loader:
#     print(batch)
#     print(batch.x.shape)
#     print(batch.edge_index.shape)
#     print(batch.other_features.shape)
#     print(batch.y.shape)
#     print(batch.batch) # This tensor maps each node to its respective graph in the batch
#     break

In [28]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool, GATConv, GINConv # Example GNN layers

class GNNPredictor(nn.Module):
    def __init__(self, node_feature_dim, edge_feature_dim, other_feature_dim, hidden_dim, output_dim=1, n_gnn_layers=3, dropout_rate=0.4):
        super(GNNPredictor, self).__init__()
        self.n_gnn_layers = n_gnn_layers
        self.dropout_rate = dropout_rate

        # GNN Layers
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList() # Batch Norm for GNN layers

        # First GNN layer
        self.convs.append(GCNConv(node_feature_dim, hidden_dim)) # Or GATConv, GINConv etc.
        self.bns.append(nn.BatchNorm1d(hidden_dim))

        # Subsequent GNN layers
        for _ in range(n_gnn_layers - 1):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
            self.bns.append(nn.BatchNorm1d(hidden_dim))

        # Calculate combined feature dimension after GNN and concatenation
        combined_feature_dim = hidden_dim + other_feature_dim # After global pooling

        # Fully Connected Layers for combined features
        self.fc1 = nn.Linear(combined_feature_dim, hidden_dim * 2)
        self.fc_bn1 = nn.BatchNorm1d(hidden_dim * 2)
        self.fc2 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc_bn2 = nn.BatchNorm1d(hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, output_dim)


    def forward(self, data):
        x, edge_index, edge_attr, other_features, batch = data.x, data.edge_index, data.edge_attr, data.other_features, data.batch

        # GNN part
        for i in range(self.n_gnn_layers):
            x = self.convs[i](x, edge_index, edge_weight=None) # Add edge_attr if layer supports it, e.g. GATConv can use edge_attr in attention
                                                               # GCNConv typically doesn't use edge_attr directly in its basic form.
                                                               # If using edge_attr, ensure GCNConv is modified or use a different conv layer.
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)

        # Global pooling (e.g., mean pooling) to get a graph-level embedding
        graph_embedding = global_mean_pool(x, batch) # `batch` vector is crucial here

        # Ensure other_features has the correct shape for concatenation
        # other_features is already [batch_size, num_other_features] from DataLoader
        # graph_embedding will be [batch_size, hidden_dim]

        # Concatenate graph embedding with other features
        combined_features = torch.cat([graph_embedding, other_features], dim=1)

        # Fully Connected Layers part
        out = self.fc1(combined_features)
        out = self.fc_bn1(out)
        out = F.relu(out)
        out = F.dropout(out, p=self.dropout_rate, training=self.training)

        out = self.fc2(out)
        out = self.fc_bn2(out)
        out = F.relu(out)
        out = F.dropout(out, p=self.dropout_rate, training=self.training)

        out = self.fc_out(out)
        return out

# --- Determine feature dimensions ---
# Get a sample data to infer dimensions (important!)
if not train_data:
    raise ValueError("Training data is empty. Cannot infer feature dimensions.")

sample_data = train_data[0]
node_dim = sample_data.x.shape[1]
edge_dim = sample_data.edge_attr.shape[1] if sample_data.edge_attr is not None and sample_data.edge_attr.ndim > 1 else 0 # Handle no edge features
other_dim = sample_data.other_features.shape[1]

hidden_channels = 128 # Hyperparameter
num_gnn_layers = 3    # Hyperparameter
dropout = 0.3        # Hyperparameter

model = GNNPredictor(node_feature_dim=node_dim,
                     edge_feature_dim=edge_dim, # Pass if your conv layer uses it
                     other_feature_dim=other_dim,
                     hidden_dim=hidden_channels,
                     output_dim=1, # Predicting a single value (inhibition)
                     n_gnn_layers=num_gnn_layers,
                     dropout_rate=dropout)

print(model)
# Test a forward pass with a batch (ensure dimensions match)
# for batch_data in train_loader:
#     try:
#         output = model(batch_data)
#         print("Forward pass successful. Output shape:", output.shape)
#         print("Target shape:", batch_data.y.shape)
#     except Exception as e:
#         print(f"Error during forward pass: {e}")
#         print("Batch data details:")
#         print(f"  x shape: {batch_data.x.shape}, type: {batch_data.x.dtype}")
#         print(f"  edge_index shape: {batch_data.edge_index.shape}, type: {batch_data.edge_index.dtype}")
#         if batch_data.edge_attr is not None:
#             print(f"  edge_attr shape: {batch_data.edge_attr.shape}, type: {batch_data.edge_attr.dtype}")
#         else:
#             print(f"  edge_attr is None")
#         print(f"  other_features shape: {batch_data.other_features.shape}, type: {batch_data.other_features.dtype}")
#         print(f"  batch tensor: {batch_data.batch.shape}, type: {batch_data.batch.dtype}")
#     break

GNNPredictor(
  (convs): ModuleList(
    (0): GCNConv(6, 128)
    (1-2): 2 x GCNConv(128, 128)
  )
  (bns): ModuleList(
    (0-2): 3 x BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc1): Linear(in_features=139, out_features=256, bias=True)
  (fc_bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc_bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_out): Linear(in_features=128, out_features=1, bias=True)
)


In [80]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Loss function (Mean Squared Error for regression)
criterion = nn.MSELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4) # Added weight decay for regularization
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=10, verbose=True) # Learning rate scheduler


def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y.unsqueeze(1)) # Ensure target y has same shape as output
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs # data.num_graphs gives the number of graphs in the batch
    return total_loss / len(loader.dataset)

@torch.no_grad() # Decorator for no gradient calculation during evaluation
def evaluate_epoch(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_preds = []
    all_targets = []
    for data in loader:
        data = data.to(device)
        out = model(data)
        loss = criterion(out, data.y.unsqueeze(1))
        total_loss += loss.item() * data.num_graphs
        all_preds.append(out.cpu())
        all_targets.append(data.y.unsqueeze(1).cpu())

    avg_loss = total_loss / len(loader.dataset)
    # You can also calculate other metrics like R2, MAE here
    # preds_tensor = torch.cat(all_preds, dim=0)
    # targets_tensor = torch.cat(all_targets, dim=0)
    # r2 = r2_score(targets_tensor.numpy(), preds_tensor.numpy()) # from sklearn.metrics
    return avg_loss


num_epochs = 100 # Example: increase for real training
best_val_loss = float('inf')

print("\nStarting Training...")
for epoch in range(1, num_epochs + 1):
    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    val_loss = evaluate_epoch(model, val_loader, criterion)
    scheduler.step(val_loss) # Step scheduler based on validation loss

    print(f"Epoch {epoch:03d}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, LR: {optimizer.param_groups[0]['lr']:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_gnn_model.pth')
        print(f"Saved new best model with Val Loss: {best_val_loss:.4f}")

print("Training finished.")

/home/nilabjab/miniconda3/envs/pyenv/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



Starting Training...
Epoch 001: Train Loss: 172.4209, Val Loss: 153.7553, LR: 0.001000
Saved new best model with Val Loss: 153.7553
Epoch 002: Train Loss: 160.1925, Val Loss: 146.3849, LR: 0.001000
Saved new best model with Val Loss: 146.3849
Epoch 003: Train Loss: 153.5186, Val Loss: 142.1578, LR: 0.001000
Saved new best model with Val Loss: 142.1578
Epoch 004: Train Loss: 149.8162, Val Loss: 132.9507, LR: 0.001000
Saved new best model with Val Loss: 132.9507
Epoch 005: Train Loss: 145.0792, Val Loss: 128.1507, LR: 0.001000
Saved new best model with Val Loss: 128.1507
Epoch 006: Train Loss: 141.7345, Val Loss: 124.5425, LR: 0.001000
Saved new best model with Val Loss: 124.5425
Epoch 007: Train Loss: 139.1298, Val Loss: 122.9401, LR: 0.001000
Saved new best model with Val Loss: 122.9401
Epoch 008: Train Loss: 136.5693, Val Loss: 119.2809, LR: 0.001000
Saved new best model with Val Loss: 119.2809
Epoch 009: Train Loss: 134.8978, Val Loss: 118.0033, LR: 0.001000
Saved new best model wit

In [32]:
from sklearn.metrics import mean_squared_error, r2_score

# Load the best model
model.load_state_dict(torch.load('best_gnn_model.pth'))
model = model.to(device) # Ensure model is on the correct device

test_loss = evaluate_epoch(model, test_loader, criterion)
print(f"\nTest Set Evaluation:")
print(f"Test MSE Loss: {test_loss:.4f}")

# Get predictions and true values for more detailed metrics
model.eval()
all_preds_test = []
all_targets_test = []
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        out = model(data)
        all_preds_test.append(out.cpu())
        all_targets_test.append(data.y.unsqueeze(1).cpu())

preds_test_tensor = torch.cat(all_preds_test, dim=0).squeeze().numpy()
targets_test_tensor = torch.cat(all_targets_test, dim=0).squeeze().numpy()

# Calculate metrics
rmse_test = np.sqrt(mean_squared_error(targets_test_tensor, preds_test_tensor))
r2_test = r2_score(targets_test_tensor, preds_test_tensor)

print(f"Test RMSE: {rmse_test:.4f}")
print(f"Test R-squared: {r2_test:.4f}")

/tmp/ipykernel_3692708/3052161693.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_gnn_model.pth'))



Test Set Evaluation:
Test MSE Loss: 68.1813
Test RMSE: 8.2572
Test R-squared: 0.6298
